<a href="https://colab.research.google.com/github/cozy-hn/Dacon-Basic-car-price-regression/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_Basic_%EC%9E%90%EB%8F%99%EC%B0%A8_%EA%B0%80%EA%B2%A9_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
import pandas as pd
import numpy as np
import random
import os

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(4242)

In [ ]:
path='/content/drive/MyDrive/데이콘 Basic 자동차 가격 예측 AI 경진대회/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

# 데이터를 확인하기 위해 head() 함수를 사용합니다.
train.head(5)

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,TRAIN_00000,2018,2014,skoda,fabia,KAT,SLA,85231,999,0,0,1,0,0,51.74
1,TRAIN_00001,2010,2006,toyota,auris,RKO,SWI,135000,1598,0,0,1,0,0,41.47
2,TRAIN_00002,2002,2002,mercedes-benz,clk-klasa,GNI,WIE,255223,1796,0,0,1,0,0,17.81
3,TRAIN_00003,2006,2001,nissan,x-trail,EHX,WIE,238000,2184,0,1,0,0,0,18.20
4,TRAIN_00004,2007,2007,fiat,bravo,OSW,MAL,251000,1910,0,1,0,0,0,17.55


In [ ]:
train.describe(include='all')

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
count,57920,57920.000000,57920.000000,57920,57920,57920,57920,5.792000e+04,57920.000000,57920.000000,57920.000000,57920.000000,57920.000000,57920.000000,57920.000000
unique,57920,NaN,NaN,20,143,3224,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,TRAIN_00000,NaN,NaN,opel,astra,WAR,MAZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,6651,2537,3245,10846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2011.355352,2008.033408,NaN,NaN,NaN,NaN,1.678691e+05,1788.160135,0.000483,0.425242,0.517956,0.010290,0.046029,52.211752
std,NaN,5.174108,5.087963,NaN,NaN,NaN,NaN,8.319120e+04,545.413818,0.021982,0.494384,0.499682,0.100918,0.209550,35.399718
min,NaN,1978.000000,1983.000000,NaN,NaN,NaN,NaN,1.000000e+00,875.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.170000
25%,NaN,2008.000000,2004.000000,NaN,NaN,NaN,NaN,1.114345e+05,1461.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.570000
50%,NaN,2011.000000,2008.000000,NaN,NaN,NaN,NaN,1.710000e+05,1753.000000,0.000000,0.000000,1.000000,0.000000,0.000000,42.250000
75%,NaN,2016.000000,2012.000000,NaN,NaN,NaN,NaN,2.200000e+05,1995.000000,0.000000,1.000000,1.000000,0.000000,0.000000,73.970000


In [ ]:
test.describe(include='all')

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG)
count,14480,14480.000000,14480.000000,14480,14480,14480,14480,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000
unique,14480,NaN,NaN,20,141,1774,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,TEST_00000,NaN,NaN,opel,astra,WAR,MAZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,1637,645,819,2707,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2011.345580,2008.031768,NaN,NaN,NaN,NaN,167044.997583,1786.886050,0.000622,0.424655,0.520994,0.010221,0.043508
std,NaN,5.213361,5.153701,NaN,NaN,NaN,NaN,82121.611500,550.501934,0.024924,0.494308,0.499576,0.100584,0.204005
min,NaN,1985.000000,1983.000000,NaN,NaN,NaN,NaN,1.000000,875.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,2008.000000,2004.000000,NaN,NaN,NaN,NaN,110722.500000,1461.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,2011.000000,2008.000000,NaN,NaN,NaN,NaN,170000.000000,1749.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,NaN,2016.000000,2012.000000,NaN,NaN,NaN,NaN,220000.000000,1995.000000,0.000000,1.000000,1.000000,0.000000,0.000000


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57920 entries, 0 to 57919
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           57920 non-null  object 
 1   생산년도         57920 non-null  int64  
 2   모델출시년도       57920 non-null  int64  
 3   브랜드          57920 non-null  object 
 4   차량모델명        57920 non-null  object 
 5   판매도시         57920 non-null  object 
 6   판매구역         57920 non-null  object 
 7   주행거리         57920 non-null  int64  
 8   배기량          57920 non-null  int64  
 9   압축천연가스(CNG)  57920 non-null  int64  
 10  경유           57920 non-null  int64  
 11  가솔린          57920 non-null  int64  
 12  하이브리드        57920 non-null  int64  
 13  액화석유가스(LPG)  57920 non-null  int64  
 14  가격           57920 non-null  float64
dtypes: float64(1), int64(9), object(5)
memory usage: 6.6+ MB


In [ ]:
columns_to_convert = ['압축천연가스(CNG)', '경유', '가솔린', '하이브리드', '액화석유가스(LPG)']

for column in columns_to_convert:
    train[column] = train[column].astype(bool)
    test[column] = test[column].astype(bool)

In [ ]:
columns_to_convert = ['생산년도', '모델출시년도', '배기량', '주행거리']

for column in columns_to_convert:
    train[column] = train[column].astype('int32')
    test[column] = test[column].astype('int32')


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57920 entries, 0 to 57919
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           57920 non-null  object 
 1   생산년도         57920 non-null  int32  
 2   모델출시년도       57920 non-null  int32  
 3   브랜드          57920 non-null  object 
 4   차량모델명        57920 non-null  object 
 5   판매도시         57920 non-null  object 
 6   판매구역         57920 non-null  object 
 7   주행거리         57920 non-null  int32  
 8   배기량          57920 non-null  int32  
 9   압축천연가스(CNG)  57920 non-null  bool   
 10  경유           57920 non-null  bool   
 11  가솔린          57920 non-null  bool   
 12  하이브리드        57920 non-null  bool   
 13  액화석유가스(LPG)  57920 non-null  bool   
 14  가격           57920 non-null  float64
dtypes: bool(5), float64(1), int32(4), object(5)
memory usage: 3.8+ MB


In [ ]:
# !pip install pandas_profiling

In [ ]:
# from pandas_profiling import ProfileReport


# # pandas-profiling 보고서 생성
# profile = ProfileReport(train, title='Pandas Profiling Report', minimal=True)
# profile.to_file(output_file=path+"output.html")
# profile

In [ ]:
# '판매도시' 컬럼의 유니크한 값들을 추출합니다.
unique_train_cities = train['판매도시'].unique()
unique_test_cities = test['판매도시'].unique()

# test 데이터셋에는 있지만, train 데이터셋에는 없는 도시를 찾습니다.
only_in_test = set(unique_test_cities) - set(unique_train_cities)

# 결과를 출력합니다.
print(only_in_test)
print("Len city", len(only_in_test))
test['판매도시'] = test['판매도시'].apply(lambda x: 'trash' if x in only_in_test else x)


{'TJO', 'XHS', 'TXI', 'WYU', 'KXQ', 'SZK', 'KRU', 'UEL', 'IPH', 'UOX', 'SXA', 'OTY', 'GDT', 'NQG', 'PGA', 'BOI', 'JUN', 'PRK', 'AGG', 'GXE', 'CWP', 'CGP', 'ZPO', 'GHS', 'LYY', 'RZA', 'JUQ', 'SED', 'QGH', 'MGJ', 'FWF', 'LNS', 'SBE', 'ZXL', 'SJB', 'WAV', 'VST', 'MWU', 'KBS', 'YIZ', 'ZGZ', 'DWT', 'VWV', 'KVY', 'PKN', 'GTC', 'BRV', 'SCA', 'PJA', 'MAT', 'XXY', 'ZLA', 'VHU', 'TLO', 'ZCS', 'MNU', 'DEU', 'VHY', 'WZD', 'PMV', 'QTF', 'OZB', 'RXJ', 'KNB', 'WYC', 'AAI', 'POS', 'SCD', 'CRQ', 'TRF', 'CQS', 'GCJ', 'NPU', 'HFW', 'ZRL', 'YXB', 'VQI', 'BJC', 'PTN', 'KIT', 'YGQ', 'WAM', 'SZQ', 'TAZ', 'MWZ', 'VCG', 'OXM', 'SKT', 'UIU', 'IOP', 'UCW', 'SEK', 'CXX', 'LIA', 'YOX', 'JUT', 'EBB', 'GUT', 'HNF', 'GGD', 'OYR', 'KIJ', 'MGV', 'DLE', 'TQT', 'DZP', 'ZVM', 'TFZ', 'UNL', 'KOO', 'GTX', 'FGG', 'QEP', 'SMI', 'CIC', 'XXD', 'BSR', 'PCK', 'JRT', 'CLC', 'PYU', 'RVI', 'OEB', 'GRZ', 'YAZ', 'JWM', 'KRJ', 'SCK', 'CJS', 'XEE', 'JWT', 'POW', 'NDR', 'GLG', 'GWR', 'AXK', 'WYG', 'DRR', 'RTD', 'KCZ', 'MDM', 'EGI', 'HER'

In [ ]:
# train 데이터셋에서 '판매도시' 컬럼의 값들을 세어 출력합니다.
print("In train dataset:")
print(train['판매도시'].value_counts())

# test 데이터셋에서 '판매도시' 컬럼의 값들을 세어 출력합니다.
print("\nIn test dataset:")
print(test['판매도시'].value_counts())


In train dataset:
WAR    3245
KRA    1241
WRO    1166
LOD    1141
POZ     987
       ... 
MVH       1
CIB       1
CED       1
FNQ       1
ZDZ       1
Name: 판매도시, Length: 3224, dtype: int64

In test dataset:
WAR      819
trash    324
LOD      322
WRO      319
KRA      293
        ... 
TCN        1
BPN        1
ZAD        1
CNQ        1
BOJ        1
Name: 판매도시, Length: 1475, dtype: int64


In [ ]:
# train 데이터셋에서 '판매도시'의 각 값의 등장 횟수를 계산합니다.
city_counts = train['판매도시'].value_counts()

# 등장 횟수가 100 미만인 값들을 찾아 'trash'로 변경합니다.
train['판매도시'] = train['판매도시'].apply(lambda x: 'trash' if city_counts[x] < 4 else x)

test['판매도시'] = test['판매도시'].apply(lambda x: 'trash' if x == 'trash' or city_counts[x] < 4 else x)


In [ ]:
# train 데이터셋에서 '판매도시' 컬럼의 값들을 세어 출력합니다.
print("In train dataset:")
print(train['판매도시'].value_counts())

# test 데이터셋에서 '판매도시' 컬럼의 값들을 세어 출력합니다.
print("\nIn test dataset:")
print(test['판매도시'].value_counts())


In train dataset:
WAR      3245
trash    2929
KRA      1241
WRO      1166
LOD      1141
         ... 
EBE         4
ZTG         4
QHC         4
HOX         4
LNQ         4
Name: 판매도시, Length: 1246, dtype: int64

In test dataset:
trash    860
WAR      819
LOD      322
WRO      319
KRA      293
        ... 
KIY        1
GIE        1
FQR        1
RSC        1
LAD        1
Name: 판매도시, Length: 1041, dtype: int64


In [ ]:
# '판매도시' 컬럼의 유니크한 값들을 추출합니다.
unique_train_cities = train['판매도시'].unique()
unique_test_cities = test['판매도시'].unique()

# test 데이터셋에는 있지만, train 데이터셋에는 없는 도시를 찾습니다.
only_in_test = set(unique_test_cities) - set(unique_train_cities)

# 결과를 출력합니다.
print(only_in_test)
print("Len city", len(only_in_test))

set()
Len city 0


In [ ]:
# 전체 '가격'의 평균을 계산합니다.
mean_price = train['가격'].mean()

# 각 '판매도시'의 '가격' 평균을 계산합니다.
city_mean_prices = train.groupby('판매도시')['가격'].mean()

# '판매도시'의 '가격' 평균과 전체 '가격'의 평균과의 차이를 계산하여 새로운 컬럼을 만듭니다.
train['del_mean'] = train['판매도시'].apply(lambda x: city_mean_prices[x] - mean_price)
# train 데이터에서 계산한 각 '판매도시'의 '가격' 평균을 test 데이터에 적용합니다.
test['del_mean'] = test['판매도시'].apply(lambda x: city_mean_prices[x] - mean_price if x in city_mean_prices else mean_price)


In [ ]:
# 전체 '가격'의 표준편차를 계산합니다.
std_price = train['가격'].std()

# 각 '판매도시'의 '가격' 표준편차를 계산합니다.
city_std_prices = train.groupby('판매도시')['가격'].std()

# '판매도시'의 '가격' 표준편차와 전체 '가격'의 표준편차와의 차이를 계산하여 새로운 컬럼을 만듭니다.
train['del_std'] = train['판매도시'].apply(lambda x: city_std_prices[x])

# train 데이터에서 계산한 각 '판매도시'의 '가격' 표준편차를 test 데이터에 적용합니다.
# 판매도시 값이 train 데이터에 없는 경우, 전체 '가격'의 표준편차를 적용합니다.
test['del_std'] = test['판매도시'].apply(lambda x: city_std_prices[x] if x in city_std_prices else std_price)


In [ ]:
train['주행거리']=np.sqrt(train['주행거리'])
test['주행거리']=np.sqrt(test['주행거리'])
# train['alpha']=(train['생산년도']-train['모델출시년도'])*np.sqrt(train['주행거리'])
# test['alpha']=(test['생산년도']-test['모델출시년도'])*np.sqrt(test['주행거리'])

In [ ]:
wrong_year = train[train['생산년도'] < train['모델출시년도']]
wrong_year

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격,del_mean,del_std
1048,TRAIN_01048,2015,2016,renault,megane,ROG,SLA,333.166625,1618,False,False,True,False,False,80.34,0.141105,23.035290
1175,TRAIN_01175,2007,2014,ford,mondeo,ZYW,OPO,505.964426,1997,False,True,False,False,False,20.02,0.977887,27.486165
1366,TRAIN_01366,2006,2015,ford,galaxy,PIA,MAZ,440.144295,1997,False,True,False,False,False,22.10,11.774382,31.683787
1857,TRAIN_01857,2005,2007,mitsubishi,lancer,GDA,POM,387.298335,1997,False,False,True,False,False,5.46,13.833109,39.293850
2369,TRAIN_02369,2009,2010,ford,focus,BAN,POM,438.178046,1596,False,False,True,False,False,24.57,-16.699837,25.701329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56816,TRAIN_56816,2015,2017,volvo,xc-60,TOR,KUJ,367.265571,2400,False,True,False,False,False,142.87,19.171115,40.043030
57028,TRAIN_57028,1992,1993,mercedes-benz,e-klasa,trash,WIE,263.266025,4196,False,False,True,False,False,139.10,-9.161974,30.899242
57357,TRAIN_57357,2007,2008,honda,jazz,KRA,MAL,389.867926,1246,False,False,True,False,False,20.80,2.647918,35.667472
57445,TRAIN_57445,2008,2009,mercedes-benz,e-klasa,OLS,WAR,469.148164,2143,False,True,False,False,False,60.97,3.210893,37.026790


In [ ]:
train.describe(include='all')

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격,del_mean,del_std
count,57920,57920.000000,57920.000000,57920,57920,57920,57920,57920.000000,57920.000000,57920,57920,57920,57920,57920,57920.000000,5.792000e+04,57920.000000
unique,57920,NaN,NaN,20,143,1246,17,NaN,NaN,2,2,2,2,2,NaN,NaN,NaN
top,TRAIN_00000,NaN,NaN,opel,astra,WAR,MAZ,NaN,NaN,False,False,True,False,False,NaN,NaN,NaN
freq,1,NaN,NaN,6651,2537,3245,10846,NaN,NaN,57892,33290,30000,57324,55254,NaN,NaN,NaN
mean,NaN,2011.355352,2008.033408,NaN,NaN,NaN,NaN,392.053230,1788.160135,NaN,NaN,NaN,NaN,NaN,52.211752,-3.654045e-15,32.023150
std,NaN,5.174108,5.087963,NaN,NaN,NaN,NaN,119.010916,545.413818,NaN,NaN,NaN,NaN,NaN,35.399718,1.373268e+01,7.618894
min,NaN,1978.000000,1983.000000,NaN,NaN,NaN,NaN,1.000000,875.000000,NaN,NaN,NaN,NaN,NaN,1.170000,-4.391775e+01,1.125833
25%,NaN,2008.000000,2004.000000,NaN,NaN,NaN,NaN,333.818064,1461.000000,NaN,NaN,NaN,NaN,NaN,24.570000,-9.222805e+00,27.985606
50%,NaN,2011.000000,2008.000000,NaN,NaN,NaN,NaN,413.521463,1753.000000,NaN,NaN,NaN,NaN,NaN,42.250000,-1.004014e+00,33.356996
75%,NaN,2016.000000,2012.000000,NaN,NaN,NaN,NaN,469.041576,1995.000000,NaN,NaN,NaN,NaN,NaN,73.970000,1.106866e+01,37.797438


In [ ]:
test.describe(include='all')

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),del_mean,del_std
count,14480,14480.000000,14480.000000,14480,14480,14480,14480,14480.000000,14480.000000,14480,14480,14480,14480,14480,14480.000000,14480.000000
unique,14480,NaN,NaN,20,141,1041,17,NaN,NaN,2,2,2,2,2,NaN,NaN
top,TEST_00000,NaN,NaN,opel,astra,trash,MAZ,NaN,NaN,False,False,True,False,False,NaN,NaN
freq,1,NaN,NaN,1637,645,860,2707,NaN,NaN,14471,8331,7544,14332,13850,NaN,NaN
mean,NaN,2011.345580,2008.031768,NaN,NaN,NaN,NaN,391.360533,1786.886050,NaN,NaN,NaN,NaN,NaN,0.105102,32.093539
std,NaN,5.213361,5.153701,NaN,NaN,NaN,NaN,117.825674,550.501934,NaN,NaN,NaN,NaN,NaN,13.641496,7.444816
min,NaN,1985.000000,1983.000000,NaN,NaN,NaN,NaN,1.000000,875.000000,NaN,NaN,NaN,NaN,NaN,-43.917752,1.125833
25%,NaN,2008.000000,2004.000000,NaN,NaN,NaN,NaN,332.749906,1461.000000,NaN,NaN,NaN,NaN,NaN,-9.161974,27.985606
50%,NaN,2011.000000,2008.000000,NaN,NaN,NaN,NaN,412.310563,1749.000000,NaN,NaN,NaN,NaN,NaN,-1.004014,33.356996
75%,NaN,2016.000000,2012.000000,NaN,NaN,NaN,NaN,469.041576,1995.000000,NaN,NaN,NaN,NaN,NaN,11.068660,37.833341


In [ ]:
train=train.drop("ID",axis=1)
test=test.drop("ID",axis=1)
train=train.drop("판매도시",axis=1)
test=test.drop("판매도시",axis=1)

In [ ]:
train = TabularDataset(train)
test = TabularDataset(test)

In [ ]:
test.describe()

,생산년도,모델출시년도,주행거리,배기량,del_mean,del_std
count,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000
mean,2011.345580,2008.031768,391.360533,1786.886050,0.105102,32.093539
std,5.213361,5.153701,117.825674,550.501934,13.641496,7.444816
min,1985.000000,1983.000000,1.000000,875.000000,-43.917752,1.125833
25%,2008.000000,2004.000000,332.749906,1461.000000,-9.161974,27.985606
50%,2011.000000,2008.000000,412.310563,1749.000000,-1.004014,33.356996
75%,2016.000000,2012.000000,469.041576,1995.000000,11.068660,37.833341
max,2022.000000,2021.000000,999.999500,6208.000000,68.125248,71.762761


In [ ]:
# predictor = TabularPredictor(label='가격', problem_type='regression',
#                              eval_metric='mean_absolute_error',sample_weight='balance_weight').fit(train,presets="best_quality",
#                                                                     auto_stack=True,save_space=True,
#                                                                     num_stack_levels=2, num_bag_folds=2,num_bag_sets=4,time_limit=20000)

predictor = TabularPredictor(label='가격', problem_type='regression',
                             eval_metric='mean_absolute_error',sample_weight='balance_weight').fit(train, save_space=True, time_limit=3600*3)

# predictor = TabularPredictor(label='가격', problem_type='regression',
#                              eval_metric='mean_absolute_error',sample_weight='balance_weight').fit(train,presets='good_quality', save_space=True, num_stack_levels=1, num_bag_folds=3,num_bag_sets=5, time_limit=3600*3)

# predictor = TabularPredictor(label='가격', problem_type='regression',
#                              eval_metric='mean_absolute_error',sample_weight='balance_weight').fit(train,presets='good_quality', save_space=True, time_limit=3600*3)

No path specified. Models will be saved in: "AutogluonModels/ag-20230619_144458/"
Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "AutogluonModels/ag-20230619_144458/"
AutoGluon Version:  0.8.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Apr 29 09:15:28 UTC 2023
Disk Space Avail:   78.58 GB / 115.66 GB (67.9%)
Train Data Rows:    57920
Train Data Columns: 14
Label Column: 가격
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9370.04 MB
	Train Data (Original)  Memory Usage: 13.08 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify 

[1000]	valid_set's l1: 5.86014
[2000]	valid_set's l1: 5.82783
[3000]	valid_set's l1: 5.82296


	-5.8167	 = Validation score   (-mean_absolute_error)
	24.17s	 = Training   runtime
	1.76s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 10771.51s of the 10771.49s of remaining time.


[1000]	valid_set's l1: 5.73066
[2000]	valid_set's l1: 5.67855
[3000]	valid_set's l1: 5.68679


	-5.6759	 = Validation score   (-mean_absolute_error)
	25.34s	 = Training   runtime
	1.12s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 10744.03s of the 10744.01s of remaining time.
	-6.0479	 = Validation score   (-mean_absolute_error)
	116.56s	 = Training   runtime
	0.69s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 10622.44s of the 10622.42s of remaining time.
	-5.7542	 = Validation score   (-mean_absolute_error)
	873.69s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 9748.37s of the 9748.35s of remaining time.
	-5.9824	 = Validation score   (-mean_absolute_error)
	44.15s	 = Training   runtime
	0.32s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 9697.57s of the 9697.55s of remaining time.
	-5.8457	 = Validation score   (-mean_absolute_error)
	107.28s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: XGBoost

[1000]	valid_set's l1: 5.63773


	-5.6186	 = Validation score   (-mean_absolute_error)
	14.04s	 = Training   runtime
	0.96s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 1079.9s of the 9321.38s of remaining time.
	-5.4923	 = Validation score   (-mean_absolute_error)
	0.77s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1479.49s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230619_144458/")


In [ ]:
preds = predictor.predict(test)

In [ ]:
submit = pd.read_csv(path+'sample_submission.csv')
submit['가격']=preds
submit.to_csv(path+'submission_6.csv', index=False)

In [ ]:
preds=preds.round(2)

In [ ]:
submit = pd.read_csv(path+'sample_submission.csv')
submit['가격']=preds
submit.to_csv(path+'submission_round_6.csv', index=False)